# McNemar's

In [1]:
# get true labels

import csv
import os
import codecs

data_dirpath = '/usr2/mamille2/tumblr/data/sample1k'

feature_tables_dir = os.path.join(data_dirpath, 'feature_tables')
filenames = ['reblog_features.csv', 'nonreblog_features.csv', 'ranking_labels.csv']
joined_filenames = [os.path.join(feature_tables_dir, filename) for filename in filenames]
# csv_readers = [csv.DictReader(codecs.open(filename, 'rU', 'utf-16')) for filename in joined_filenames]
csv_readers = [csv.DictReader(x.replace('\0', '') for x in open(filename, 'r')) for filename in joined_filenames]

instances = []
instance_labels = []
for row in zip(*csv_readers):
    reblog_features = row[0]
    nonreblog_features = row[1]
    label = int(row[2]['ranking_label'])
    instance = (reblog_features, nonreblog_features) # reblog always first, nonreblog always second
    instances.append(instance)
    instance_labels.append(label)
    
print(len(instances), len(instance_labels))

from sklearn import model_selection

X = []
y = []
for (reblog_candidate, nonreblog_candidate), label in zip(instances, instance_labels):
    X.append([])
    y.append(label)
    
X_train, X_test, y_train, y_test = model_selection.train_test_split(X, y, test_size=0.1, random_state=12345)

712670 712670


In [2]:
identity_categories = ['age', 'ethnicity/nationality', 'fandoms', 'gender',
                       'interests', 'location', 'personality type', 'pronouns', 'relationship status', 'roleplay',
                       'sexual orientation', 'zodiac']
len(identity_categories)

12

In [ ]:
import numpy as np
import os
data_dirpath = '/usr2/mamille2/tumblr/data/sample1k'

for category in identity_categories + ['all']:
    print(category)
    baseline_pred = np.loadtxt(os.path.join(data_dirpath, 'output', 'predictions', f'lr_baseline+exp1_{category.replace("/", "_").replace(" ", "_")}_test_preds.txt'))
    experiment_pred = np.loadtxt(os.path.join(data_dirpath, 'output', 'predictions', f'lr_baseline+exp2_{category.replace("/", "_").replace(" ", "_")}_test_preds.txt'))
#     print(baseline_pred.shape)
#     print(experiment_pred.shape)

    a = 0
    b = 0 # Baseline correct, experiment incorrect
    c = 0 # Baseline incorrect, experiment correct
    d = 0
    for b_pred, ex_pred, true in zip(baseline_pred, experiment_pred, y_test):
        if b_pred == true and ex_pred == true:
            a += 1
        elif b_pred == true and ex_pred != true:
            b += 1
        elif b_pred != true and ex_pred == true:
            c += 1
        else:
            d += 1

    table = [[a, b],
             [c, d]]
#     print(table)

    # Example of calculating the mcnemar test
    from statsmodels.stats.contingency_tables import mcnemar
    # calculate mcnemar test
    result = mcnemar(table, exact=False, correction=False)
    # summarize the finding
    print('statistic=%.3f, p-value=%.6f' % (result.statistic, result.pvalue))
    # interpret the p-value
    alpha = 0.05
    if result.pvalue > alpha:
        print('Same proportions of errors (fail to reject H0)')
    else:
        print('Different proportions of errors (reject H0)')
    print()

age
statistic=89.030, p-value=0.000000
Different proportions of errors (reject H0)

ethnicity/nationality
statistic=16.680, p-value=0.000044
Different proportions of errors (reject H0)

fandoms
statistic=1.388, p-value=0.238725
Same proportions of errors (fail to reject H0)

gender
statistic=62.811, p-value=0.000000
Different proportions of errors (reject H0)

interests
statistic=98.539, p-value=0.000000
Different proportions of errors (reject H0)

location
statistic=22.172, p-value=0.000002
Different proportions of errors (reject H0)

personality type
statistic=0.000, p-value=1.000000
Same proportions of errors (fail to reject H0)

pronouns
